<a href="https://colab.research.google.com/github/almutareb/InnovationPathfinderAI/blob/agent_notebook/Copy_of_local_ollama_agent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain_community -q
!pip install -U langchain -q
!pip install google-search-results -q
!pip install langchainhub -q
!pip install text_generation -q
!pip install arxiv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [7]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(
    model="mistral",
    base_url="https://0013-35-201-206-176.ngrok-free.app"
    )
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# https://python.langchain.com/docs/expression_language/why
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
print(chain.invoke({"topic": "Space travel"}))

 Why don't astronauts forget things in space?

Because in space, they weightless information! 💡🚀

(This one is an oldie but goodie!)


In [8]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import SerpAPIWrapper
from langchain.retrievers import ArxivRetriever
from langchain_core.tools import Tool

retriever = ArxivRetriever(load_max_docs=2)

tools = [create_retriever_tool(
    retriever,
    "search arxiv's database for",
    "Use this to recomend the user a paper to read Unless stated please choose the most recent models",
    # "Searches and returns excerpts from the 2022 State of the Union.",
),

Tool(
    name="SerpAPI",
    description="A low-cost Google Search API. Useful for when you need to answer questions about current events. Input should be a search query.",
    func=SerpAPIWrapper().run,
)

]

# tools = [tool]

In [9]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description
# from langchain_community.utilities import SerpAPIWrapper
# from langchain.retrievers import ArxivRetriever

# setup tools
# tools = load_tools(["serpapi", "llm-math"], llm=llm)

# tools = load_tools(["serpapi", "llm-math",create_retriever_tool], llm=llm)

# setup ReAct style prompt
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

chat_model = llm
# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

# agent_executor.invoke(
#     {
#         "input": "Who is the current holder of the speed skating world record on 500 meters? What is her current age raised to the 0.43 power?"
#     }
# )

# agent_executor.invoke(
#     {
#         "input": "what are large language models and why are they so expensive to run?"
#     }
# )

agent_executor.invoke(
    {
        "input": "How to generate videos from images using state of the art macchine learning models"
    }
)






> Entering new AgentExecutor chain...
 Thought: To find the most recent papers on generating videos from images using machine learning models, I will use the "search arxiv's database for" tool.

Action:
```json
{
  "action": "search arxiv's database for",
  "action_input": "generating videos from images machine learning"
}
```Diffusion models exhibited tremendous progress in image and video generation,
exceeding GANs in quality and diversity. However, they are usually trained on
very large datasets and are not naturally adapted to manipulate a given input
image or video. In this paper we show how this can be resolved by training a
diffusion model on a single input image or video. Our image/video-specific
diffusion model (SinFusion) learns the appearance and dynamics of the single
image or video, while utilizing the conditioning capabilities of diffusion
models. It can solve a wide array of image/video-specific manipulation tasks.
In particular, our model can learn from few frames the

{'input': 'How to generate videos from images using state of the art macchine learning models',
 'output': 'There are recent studies that discuss generating videos from images using machine learning models. One study introduces SinFusion, a diffusion model for image/video-specific manipulation tasks. Another study deals with high-level event detection in videos using textual descriptions and related videos. The third study proposes a novel video-to-video synthesis approach under the generative adversarial learning framework to achieve photorealistic, temporally coherent videos.'}